In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

Using TensorFlow backend.


In [2]:
# The data, split between train and test sets:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
print('y_train shape:', y_train.shape)


y_train shape: (50000, 10)


In [4]:
from fedml import Alliance, AllianceMember,KerasSequentialBaseLearner, FedAveragingClassifier
from read_data import read_training_data
import numpy as np

alliance = Alliance()
M=10


    

def split_data_noniid(x,y,fractions):
    s = len(x)
    l = np.eye(10)
    sorted_ind=np.array([np.where([list(y_)==list(l[i]) for y_ in y])[0] for i in range(10)])
    
    parts=2
    c = np.array([np.int32(np.linspace(0,len(sorted_ind[i]),parts+1)) for i in range(10)])
    new_sorted_ind = np.array([[sorted_ind[i][c[i,j]:c[i,j+1]] for j in range(parts)] for i in range(10)])
   
    m = 10
    x_list = np.empty(10,list)
    y_list = np.empty(10,list)
    
    for i in range(10):
        if i<9:
            j=i+1
        else:
            j=0   
        x_list[i] = np.concatenate((x[new_sorted_ind[i,0]],x[new_sorted_ind[j,1]]))
        y_list[i] = np.concatenate((y[new_sorted_ind[i,0]],y[new_sorted_ind[j,1]]))
        new_order = np.random.permutation(len(x_list[i]))
        x_list[i] = x_list[i][new_order]
        y_list[i] = y_list[i][new_order]

    

    return np.array(x_list),np.array(y_list)



#F = [10,10,8,8,6,6,4,4,2,2,1,1]
#M = len(F)

x_train_splits, y_train_splits = split_data_noniid(x_train,y_train,fractions=10)
print("x_train_splits shape: ", x_train_splits.shape)
for x, y in zip(x_train_splits,y_train_splits):
    print("x shape: ", x.shape, ", y shape: ", y.shape)
#x_test_splits, y_test_splits = split_data(x_test,y_test,fractions=F)
#print("x_train shape: ", np.array(x_train).shape)
#tot_len = 0
#for split in x_train_splits:
#    print("x_train_splits shape: ", split.shape)
#    tot_len += len(split)
#print("tot len: ", tot_len)
alliance.set_validation_dataset(x_test,y_test)

classes = np.arange(10)
for i in range(M):
    member = AllianceMember(x_train_splits[i], y_train_splits[i], classes=classes)
    alliance.add_member(member)

x_train_splits shape:  (10,)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)


In [5]:
i=0
for y in y_train_splits:
    t = []
    for y_ in y:
        t += [np.argmax(y_)]
    print("Classes for member ", i, ":", np.unique(np.array(t)))
    if i == 0:
        print("t: ")
        for j in range(100):
            print(t[j])
    i+=1

Classes for member  0 : [0 1]
t: 
0
0
0
0
1
0
1
0
1
0
0
0
0
1
1
0
1
0
0
0
0
1
1
0
1
0
0
0
1
0
0
1
1
1
1
0
1
1
0
1
0
1
0
1
1
1
1
1
0
1
0
0
1
1
1
1
0
1
1
0
0
1
0
1
1
0
0
0
1
0
0
0
0
0
1
0
1
0
0
1
0
0
1
0
1
0
0
0
0
1
0
1
0
1
0
1
1
0
1
0
Classes for member  1 : [1 2]
Classes for member  2 : [2 3]
Classes for member  3 : [3 4]
Classes for member  4 : [4 5]
Classes for member  5 : [5 6]
Classes for member  6 : [6 7]
Classes for member  7 : [7 8]
Classes for member  8 : [8 9]
Classes for member  9 : [0 9]


In [6]:
from cifarmodel import KerasSequentialCifar
cifarmodelparameters= {"optimizer": keras.optimizers.Adam, "learning_rate": 0.1, "decay": 0}
base_learner = KerasSequentialCifar(cifarmodelparameters)
fedavg_model = FedAveragingClassifier(alliance=alliance,base_learner=base_learner)


TypeError: __init__() takes 1 positional argument but 2 were given

In [ ]:
parameters = {"nr_global_iterations": 100, "nr_local_iterations":1, "training_steps": None, "data_augmentation": True, "model_size_averaging": True} 
training_loss, test_loss = fedavg_model.fit(parameters)

In [ ]:
from matplotlib import pylab as plt
import pickle
pickle.dump(fedavg_model.test_loss,open('test_loss_noniid_1epochloc.p','wb'))


plt.plot(fedavg_model.test_loss)
plt.savefig('accuracy_progress_noniid_1epochloc')
plt.show()
